In [3]:
import os
import pandas as pd
import numpy as np
from uuid import uuid4
import pickle

In [4]:
n,m = 1000, 20
frame = pd.DataFrame(data=np.random.randn(n, m), columns=[str(uuid4()) for _ in range(0, m)])

## Attempts without Mongo

In [5]:
%%timeit
frame.to_parquet("test.parquet", engine="pyarrow")
pd.read_parquet("test.parquet", engine="pyarrow")

9.92 ms ± 317 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit
frame.to_pickle("test.pickle")
pd.read_pickle("test.pickle")

1.33 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [48]:
%%timeit
# not that here is no i/o to disk
x = Binary(pickle.dumps(frame))
pickle.loads(x)

509 µs ± 53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Connecting to Mongo

In [28]:
MONGODB_SETTINGS = {'db': 'mongo', 'host': 'mongo'}

from mongoengine import *
connect(**MONGODB_SETTINGS)

MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## Using Parquet, Convert frame to parquet and back

In [6]:
from antarctic.PandasFields import ParquetFrameField

class Maffay(Document):
    frame = ParquetFrameField()

In [7]:
%%timeit
m=Maffay(frame=frame).save()
m.frame

9.29 ms ± 513 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Using Pickle, Convert using Pickle and back

In [31]:
from antarctic.PandasFields import PickleFrameField

class Falco(Document):
    frame = PicklePandasField()